<a href="https://colab.research.google.com/github/etodasha/PRA-M2.875-2020.2/blob/main/PRA_DL_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 69 bytes


In [ ]:
!kaggle datasets download "jordidelatorreuoc/kaggle-plant-pathology-2021-modificat"

 98% 955M/970M [00:05<00:00, 129MB/s]
100% 970M/970M [00:05<00:00, 170MB/s]


In [ ]:
!unzip kaggle-plant-pathology-2021-modificat.zip

In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import keras 
import os
import matplotlib.pylab as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, ZeroPadding2D, GlobalAveragePooling2D,BatchNormalization
from keras.optimizers import Adam, SGD,RMSprop,Adadelta,Adagrad,Adamax,Nadam,Ftrl
from keras.layers.experimental.preprocessing import Resizing
from keras.callbacks import *

TRAIN VAL + DATA AUGMENTATION

In [ ]:
datagen_train = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8,1.2],
    zoom_range=[0.5,1.2],
    rotation_range=20,
    rescale=1./255,
    validation_split=0.2) 

train_generator = datagen_train.flow_from_directory( 
    'train',
    subset='training')

validation_generator = datagen_train.flow_from_directory(
    'train',
    subset='validation'
    ,shuffle=False)



Found 7800 images belonging to 6 classes.
Found 1950 images belonging to 6 classes.


BASE CNN MODEL

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from keras import backend as K

def custom_recall(y_true, y_pred):
  TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = TP / (Positives+K.epsilon())
  return recall 

def custom_precision(y_true, y_pred):
  TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = TP / (Pred_Positives+K.epsilon())
  return precision

def custom_f1(y_true, y_pred):    
    precision, recall = custom_precision(y_true, y_pred), custom_recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:

base_model = Sequential()
base_model.add(keras.Input(shape=(332, 498, 3)))

base_model.add(Resizing(64,64))

base_model.add(Conv2D(32, 5, padding="same", strides=(1, 1), activation="relu"))
base_model.add(MaxPool2D(pool_size=(2, 2)))

base_model.add(Conv2D(64, 5, padding="same", strides=(1, 1), activation="relu"))
base_model.add(MaxPool2D(pool_size=(2, 2)))

base_model.add(Conv2D(128, 5, padding="same", strides=(1, 1), activation="relu"))
base_model.add(MaxPool2D(pool_size=(2, 2)))

base_model.add(Flatten())

base_model.add(Dense(1024,activation="relu"))
base_model.add(Dropout(0.4))

base_model.add(Dense(6, activation="softmax"))

base_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 32)        2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       204928    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0

In [ ]:
base_model.compile(optimizer = Adam(learning_rate=0.0001), 
              loss = "categorical_crossentropy", 
              metrics = ['accuracy',custom_f1,custom_recall,custom_precision])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_custom_f1', mode='max', patience=25, 
                                               restore_best_weights=True, verbose=2)

filepath = "/content/gdrive/My Drive/epochs:{epoch:03d}-val_f1:{val_custom_f1:.3f}.hdf5"

checkpoint_callback = ModelCheckpoint(filepath= filepath, monitor='val_custom_f1', mode='max',
                                                         save_weights_only=True, save_best_only=True)


base_history = base_model.fit(train_generator,
                              validation_data = validation_generator,
                              epochs=200,
                              batch_size=64,
                              callbacks=[early_stopping,checkpoint_callback],
                              verbose=1) 

Epoch 1/200
244/244 [==============================] - 361s 1s/step - loss: 1.5814 - accuracy: 0.4139 - custom_f1: 0.1311 - custom_recall: 0.0892 - custom_precision: 0.3924 - val_loss: 1.3955 - val_accuracy: 0.4990 - val_custom_f1: 0.2713 - val_custom_recall: 0.1983 - val_custom_precision: 0.4510
Epoch 2/200
244/244 [==============================] - 344s 1s/step - loss: 1.3980 - accuracy: 0.4939 - custom_f1: 0.3437 - custom_recall: 0.2430 - custom_precision: 0.6301 - val_loss: 1.3431 - val_accuracy: 0.5185 - val_custom_f1: 0.3424 - val_custom_recall: 0.2587 - val_custom_precision: 0.5448
Epoch 3/200
244/244 [==============================] - 340s 1s/step - loss: 1.3749 - accuracy: 0.4925 - custom_f1: 0.3516 - custom_recall: 0.2535 - custom_precision: 0.6163 - val_loss: 1.3429 - val_accuracy: 0.5149 - val_custom_f1: 0.3434 - val_custom_recall: 0.2551 - val_custom_precision: 0.5372
Epoch 4/200
244/244 [==============================] - 341s 1s/step - loss: 1.3453 - accuracy: 0.4991 - cu

In [ ]:
# base 0.759